# Bi-MIChI UM Experiement

In this notebook we show the synthetic "UM" experiement that was presented in the paper. The goal is to positively detect both the "U" and the "M" in the final fused image. For more information, please refer to the Bi-MIChI paper.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from skimage.segmentation import slic
from bicap_train import *
from utils import normalize_to_range, mark_bags

cmap = 'bwr'
vmin = -1
vmax = 1

## Configure Dataset

First, we load the data and construct the ground truths. There are 2 ground truths - one for each objective function we have. 

In [ ]:
# load the image
img = cv2.imread("um-test/um.bmp", 0)
bin_img = np.array(img, dtype=np.float64)
bin_img[bin_img > 0] = 1
bin_img[bin_img == 0] = -1
bin_img[bin_img == 1] = 0
bin_img[bin_img == -1] = 1

fig = plt.figure()
grid = ImageGrid(fig, 111,          # as in plt.subplot(111)
                 nrows_ncols=(1,2),
                 axes_pad=0.15,
                 share_all=True,
                 cbar_location="right",
                 cbar_mode="single",
                 cbar_size="7%",
                 cbar_pad=0.15
                )

# obj function 1 ground truth
um_gt1 = bin_img.copy()
um_gt1 = normalize_to_range(um_gt1, min=-1, max=1)
grid[0].imshow(um_gt1, cmap=cmap, vmin=vmin, vmax=vmax)
grid[0].set_title("GT - objective function 1\n(without 0 bound)")

# obj function 2 ground truth
um_gt2 = bin_img.copy()
um_gt2 = normalize_to_range(um_gt2, min=0, max=1)
plt2 = grid[1].imshow(um_gt2, cmap=cmap, vmin=vmin, vmax=vmax)
grid[1].set_title("GT - objective function 2\n(with 0 bound)")

grid[1].cax.colorbar(plt2)
plt.show()
 

Next, we construct the three sources to be fused:
- Source 1 positvely detects the "U" and negatively detects everything else.
- Source 2 negatively detects the "M" and positively detects everything else.
- Source 3 negatively detects the "UM" and positively detects the background.

In [ ]:
fig = plt.figure()
grid = ImageGrid(fig, 111,          # as in plt.subplot(111)
                 nrows_ncols=(1,3),
                 axes_pad=0.15,
                 share_all=True,
                 cbar_location="right",
                 cbar_mode="single",
                 cbar_size="7%",
                 cbar_pad=0.15
                )



source1 = um_gt2.copy()
source1[40:, :] = 0
source1 = normalize_to_range(source1, min=-1, max=1)
grid[0].imshow(source1, cmap=cmap, vmin=vmin, vmax=vmax)
grid[0].set_title("Source 1")

source2 = um_gt2.copy()
source2[source2 == 1] = -1
source2[:40, :] = 0
source2 = normalize_to_range(source2, min=-1, max=1)
grid[1].imshow(source2, cmap=cmap, vmin=vmin, vmax=vmax)
grid[1].set_title("Source 2")

source3 = um_gt2.copy()
source3[source3 == 1] = -1
source3 = normalize_to_range(source3, min=-1, max=1)
plt3 = grid[2].imshow(source3, cmap=cmap, vmin=vmin, vmax=vmax)
grid[2].set_title("Source 3")

grid[2].cax.colorbar(plt3)
plt.show()

## Construct Data Bags

We use the SLIC superpixel algorithm to construct positive and negative bags from the ground truth.

In [ ]:
n_seg = 50
compactness = 1.0

seg = slic(um_gt1, n_segments=n_seg, compactness=compactness, channel_axis=None)
n_seg = seg.max()

data = np.empty(n_seg, dtype=object)
labels = np.empty(n_seg)
ind = 0
for i in range(1, n_seg + 1):
    s1 = source1[seg == i].reshape((-1, 1))
    s2 = source2[seg == i].reshape((-1, 1))
    s3 = source3[seg == i].reshape((-1, 1))

    comb = np.hstack((s1, s2, s3))
    data[ind] = comb
    labels[ind] = np.any(s1 == 1) or np.any(s2 == -1) or np.any(s3 == -1)

    ind += 1

boundaries = mark_bags(um_gt1, seg, labels)
plt.imshow(boundaries)
plt.title("Labeled Bags (green=positive, red=negative)")
plt.show()

## Bi-MIChI Optimization - Objective Function 1

The first optimization we run utilizes objective function 1, which **does not** enforce the 0 bound of the bi-capacities.



In [ ]:
params = {
    "max_iter": 5000, 
    "eta": 0.8,
    "pop_size": 8,
    #"n_threads": 32,
    "fitness_thresh": 0.01,
    "use_zero_bound": False 
}
bicap_train = BicapEvolutionaryTrain(data, labels, params)
bicap = bicap_train.train()
print(bicap)

out_img = np.zeros_like(um_gt1)
ind = 0
for i in range(1, n_seg + 1):
    ci = choquet_integral(data[ind], bicap)
    out_img[seg == i] = ci.reshape(-1)
    ind += 1

plt.imshow(out_img, cmap=cmap, vmin=vmin, vmax=vmax)
plt.title("Fusion Output - obj function 1")
plt.show()

## Bi-MIChI Optimization - Objective Function 2

The second optimization we run utilizes objective function 2, which **does** enforce the 0 bound of the bi-capacities.

In [ ]:
params = {
    "max_iter": 5000, 
    "eta": 0.6,
    "pop_size": 8,
    #"n_threads": 32,
    "fitness_thresh": 0.01,
    "use_zero_bound": True
}
bicap_train = BicapEvolutionaryTrain(data, labels, params)
bicap = bicap_train.train()
print(bicap)

out_img = np.zeros_like(um_gt1)
ind = 0
for i in range(1, n_seg + 1):
    ci = choquet_integral(data[ind], bicap)
    out_img[seg == i] = ci.reshape(-1)
    ind += 1

plt.imshow(out_img, cmap=cmap, vmin=vmin, vmax=vmax)
plt.title("Fusion Output - obj function 2")
plt.show()

out_img_abs = np.abs(out_img)
plt.imshow(out_img_abs, cmap=cmap, vmin=vmin, vmax=vmax)
plt.title("Fusion Output - obj function 2 (abs)")
plt.show()